Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [3]:
%load_ext sql
%sql sqlite:///HomeW.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @HomeW.db'

In [54]:
%%sql
PRAGMA foreign_keys=ON;
drop table if exists PlayersArchive;
drop table if exists Players;
drop table if exists Teams;
drop table if exists Coaches;
drop table if exists Positions;


create table Coaches (
    id integer not null primary key,
    name varchar(30)
);
create table Positions(
    id integer not null primary key,
    name varchar(30)
);
create table Teams(
    id integer not null primary key,
    name varchar(30),
    location varchar(30),
    coach_id integer,
    foreign key(coach_id) references Coaches(id)
);
create table Players(
    id integer not null primary key,
    name varchar(30),
    age integer,
    position_id integer,
    team_id integer,
    start_date date,
    foreign key(position_id) references Positions(id)
);

create table PlayersArchive(
    player_id integer,
    team_id integer,
    start_date date not null,
    end_date date,
    foreign key(player_id) references Players(id),
    foreign key(team_id) references Teams(id),
    primary key(player_id,team_id,start_date)
);


 * sqlite:///HomeW.db
Done.
(sqlite3.OperationalError) database is locked [SQL: 'drop table if exists PlayersArchive;'] (Background on this error at: http://sqlalche.me/e/e3q8)


# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [55]:
%%sql
insert into Positions (name) values ("Goalkeeper"),("Full-back"),("Wing-back"),("Forward")

 * sqlite:///HomeW.db
(sqlite3.OperationalError) database is locked (Background on this error at: http://sqlalche.me/e/e3q8)


# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [3]:
import sqlite3

In [4]:
def add_coach(name):
    db=sqlite3.connect('HomeW.db')
    
    cursor = db.cursor()
    cursor1 = db.cursor()
    cursor1.execute("select id from Coaches where name == ?",(name,))
    try:
        r = cursor1.fetchone()[0]
        print("Table contains this Coach with id = ",r)
    except TypeError:
        cursor.execute("insert into Coaches(name) values(?)",(name,))
        db.commit()
    db.close()
    

In [5]:
add_coach("Coach_0")
add_coach("Coach_2")
add_coach("Coach_3")

Table contains this Coach with id =  1
Table contains this Coach with id =  2
Table contains this Coach with id =  3


In [6]:
def print_coaches():
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select * from Coaches"
    cursor.execute(query)
    try:
        for item in cursor.fetchall():
            print(item)
        return
        pass
    except:       
        pass
    db.close()
    

In [7]:
print_coaches()

(1, 'Coach_0')
(2, 'Coach_2')
(3, 'Coach_3')


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [8]:
def add_team(name,location,coach_name):
    db=sqlite3.connect('HomeW.db')
    cursor_1 = db.cursor()
    query = "select * from Teams where name = '{0}'".format(name)
    cursor_1.execute(query)
    try:
        data = cursor_1.fetchone()[0]
        print("team exists")
        return
        pass
    except:       
        pass
    cursor = db.cursor()
    query = "insert into Teams(name,location,coach_id) values('{0}','{1}',(select id from Coaches where name = '{2}'))".format(name,location,coach_name)
    cursor.execute(query)
    db.commit()
    db=sqlite3.connect('HomeW.db')

In [9]:
add_team("team_1","Moscow","Coach_0")
add_team("team_2","Moscow","Coach_2")
add_team("team_3","Moscow","Coach_3")

team exists
team exists
team exists


In [10]:
def print_teams():
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select * from Teams"
    cursor.execute(query)
    try:
        for item in cursor.fetchall():
            print(item)
        return
        pass
    except:       
        pass
    db.close()
    

In [11]:
print_teams()

(1, 'team_1', 'New York', 1)
(2, 'team_2', 'Moscow', 2)
(3, 'team_3', 'Moscow', 3)


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [12]:
def update_team(team_name,coach_name=None,location=None):
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    if coach_name != None:
        query = "update Teams set coach_id = (select id from Coaches where name = '{0}') where name = '{1}'".format(coach_name,team_name)
        cursor.execute(query)
        db.commit()
    if location != None:
        query = "update Teams set location =  '{0}' where name = '{1}' ".format(location,team_name)
        cursor.execute(query)
        db.commit()
    db.close()
    

In [13]:
update_team("team_2",coach_name="Coach_2")
update_team("team_1",location="New York")

In [14]:
print_teams()

(1, 'team_1', 'New York', 1)
(2, 'team_2', 'Moscow', 2)
(3, 'team_3', 'Moscow', 3)


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [15]:
def print_positions():
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select * from Positions"
    cursor.execute(query)
    try:
        for item in cursor.fetchall():
            print(item)
        return
        pass
    except:       
        pass
    db.close()
    

In [16]:
print_positions()

(1, 'Goalkeeper')
(2, 'Full-back')
(3, 'Wing-back')
(4, 'Forward')


In [17]:
def get_position(name):
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select id from Positions where name ='{}'".format(name)
    cursor.execute(query)
    try:
        return cursor.fetchone()[0]
    except:
        return -1;

In [18]:
def get_team_id(name):
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select id from Teams where name = '{}'".format(name)
    cursor.execute(query)
    try:
        return cursor.fetchone()[0]
    except:
        return -1;

In [19]:
def get_player_data(name):
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select team_id,start_date,id from Players where name = '{}'".format(name)
    cursor.execute(query)
    try:
        return cursor.fetchone()
    except:
        return -1;

In [21]:
def add_player(name,age,position,team_name,date=None):
    
    if(age > 10 and age <50):
        db=sqlite3.connect('HomeW.db')
        p_id = None
        t_id = None
        p_id = get_position(position)
        if p_id==-1 :
            print("Wrong position")
            return;
        t_id = get_team_id(team_name)
        if t_id ==-1 :
            print("Wrong team")
            return;
        data = get_player_data(name)
        if data == -1:
            query = ""
            if date == None:
                cursor_d = db.cursor()
                cursor_d.execute("select date('now')")
                da = cursor_d.fetchone()[0]
                query= "insert into Players(name,age,position_id,team_id,start_date) values('{0}','{1}','{2}','{3}','{4}')".format(name,age,p_id,t_id,da)
            else:
                query= "insert into Players(name,age,position_id,team_id,start_date) values('{0}','{1}','{2}','{3}','{4}')".format(name,age,p_id,t_id,date)
            cursor.execute(query)
            db.commit()
            pass
        else:
            team_id = data[0]
            start_date = data[1]
            player_id = data[2]
            query = ""
            query1 = ""
            cursor1 = db.cursor()
            cursor2 = db.cursor()
            if date == None:
                cursor_d = db.cursor()
                cursor_d.execute("select date('now')")
                date1 = cursor_d.fetchone()[0]
                query= "update Players set team_id = (select id from Teams where name = '{0}'),\
                position_id = (select id from Positions where name = '{1}'),age = '{2}',start_date = '{3}' where name = '{4}'".format(team_name,position,age,date1,name)
                
                query1 = "insert into PlayersArchive(player_id,team_id,start_date,end_date) values ('{0}','{1}','{2}','{3}')".format(player_id,team_id,start_date,date1)
            else:
                query= "update Players set team_id = (select id from Teams where name = '{0}'),\
                position_id = (select id from Positions where name = '{1}'),age = '{2}',start_date = '{3}' where name = '{4}'".format(team_name,position,age,date,name)
                
                query1 = "insert into PlayersArchive(player_id,team_id,start_date,end_date) values ('{0}','{1}','{2}','{3}')".format(player_id,team_id,start_date,date)
            cursor1.execute(query)
            db.commit()
            cursor2.execute(query1)
            db.commit()
    else:
        print("Wrong Age")

In [29]:
add_player("1",11,"Goalkeeper","team_3")


In [5]:
%%sql
DELETE FROM Players;

 * sqlite:///HomeW.db
2 rows affected.


[]

In [107]:
db.close()

In [23]:
def print_players():
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select * from Players"
    cursor.execute(query)
    try:
        for item in cursor.fetchall():
            print(item)
        return
        pass
    except:       
        pass
    db.close()

In [30]:
print_players()

(1, '1', 11, 1, 3, '2019-03-13')


In [27]:
def print_players_archive():
    db=sqlite3.connect('HomeW.db')
    cursor = db.cursor()
    query = "select * from PlayersArchive"
    cursor.execute(query)
    try:
        for item in cursor.fetchall():
            print(item)
        return
        pass
    except:       
        pass
    db.close()

In [31]:
print_players_archive()

(1, 3, '2019-03-02', 'asd')
(1, 'None', '2019-03-02', '2019-03-13')
(1, 1, '2019-03-13', '2019-03-13')
(1, 2, '2019-03-13', '2019-03-13')


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [2]:
import sqlite3
def change_team(name,team_name,end_date):
    db=sqlite3.connect('HomeW.db')
    cursor1 = db.cursor()
    query1 = "select team_id,start_date,id from Players where name = '{}'".format(name)
    cursor1.execute(query1)
    team_id = ""
    start_date = ""
    player_id = ""
    try:
        data = cursor1.fetchone()
        team_id = data[0]
        start_date = data[1]
        player_id = data[2]
    except:
        print("Error")
        return
    print(start_date)
    cursor2 = db.cursor()
    query2 = "update Players set team_id = (select id from Teams where name = '{0}') where name = '{1}'".format(team_name,name)
    cursor2.execute(query2)
    try:
        pass
    except:
        pass
    cursor3= db.cursor()
    query3 = "insert into PlayersArchive(player_id,team_id,start_date,end_date) values ('{0}','{1}','{2}','{3}')".format(player_id,team_id,start_date,end_date)
    cursor3.execute(query3)
    db.commit()
    db.close()

In [3]:
change_team("1","asd","asd")

2019-03-02
